# Testing tunability of models

In [2]:
# Packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
import os
os.getcwd()

'c:\\Users\\bartekb\\Desktop\\sem5\\automl\\projekt\\code'

In [4]:
# diabetes = pd.DataFrame(pd.read_csv("C:/Users/jancw/Desktop/Studia/Sem5/AutoML/Project_1/Datasets/diabetes.csv"))
diabetes = pd.DataFrame(pd.read_csv("../data/diabetes.csv"))
students = pd.DataFrame(pd.read_csv('../data/students.csv'))
# students = pd.DataFrame(pd.read_csv('C:/Users/jancw/Desktop/Studia/Sem5/AutoML/Project_1/Datasets/students.csv'))
students = students.loc[students.Target!='Enrolled']
# telco = pd.DataFrame(pd.read_csv('C:/Users/jancw/Desktop/Studia/Sem5/AutoML/Project_1/Datasets/telco.csv'))
telco = pd.DataFrame(pd.read_csv('../data/telco.csv'))
telco['TotalCharges'] = pd.to_numeric(telco['TotalCharges'],errors='coerce')
telco = telco.iloc[:,1:] # id column
# pistachio = pd.DataFrame(pd.read_csv('C:/Users/jancw/Desktop/Studia/Sem5/AutoML/Project_1/Datasets/pistachio.csv'))
pistachio = pd.DataFrame(pd.read_csv('../data/pistachio.csv'))
datasets = {'diabetes': diabetes,'students': students,'telco': telco,'pistachio': pistachio}

# Wybór danych i rekonesans

In [5]:
# datasets sizes
for key in datasets:
    print(f'{key} len: {len(datasets.get(key))}')

diabetes len: 100000
students len: 3630
telco len: 7043
pistachio len: 2148


In [6]:
# we need about 1500 rows from each set
random_seed = 24
datasets_wanted_size = {}
for key in datasets:
    dataset_size = len(datasets.get(key))
    X = datasets.get(key)
    y = datasets.get(key).iloc[:,-1]
    rest,data_wanted_size = train_test_split(X,test_size=1500/dataset_size,stratify=y,random_state=random_seed)
    datasets_wanted_size[key] = data_wanted_size

In [7]:
# typy kolumn
for key in datasets_wanted_size:
    print(key + ':')
    print(f'{datasets_wanted_size.get(key).dtypes.value_counts()}\n')

diabetes:
int64      4
float64    3
object     2
Name: count, dtype: int64

students:
int64      29
float64     5
object      1
Name: count, dtype: int64

telco:
object     16
int64       2
float64     2
Name: count, dtype: int64

pistachio:
float64    26
int64       2
object      1
Name: count, dtype: int64



In [8]:
# rozkład y
for key in datasets_wanted_size:
    print(key+':')
    print(f'{datasets_wanted_size.get(key).iloc[:,-1].value_counts()}\n')

# zmieniamy dla każdego zbioru ,y na wartości {0,1}
label_encoder = LabelEncoder()
for key in datasets_wanted_size:
    datasets_wanted_size[key].iloc[:,-1] = label_encoder.fit_transform(datasets_wanted_size[key].iloc[:,-1])
    datasets_wanted_size[key] = datasets_wanted_size[key].astype({datasets_wanted_size.get(key).columns[-1]: 'int64'})

diabetes:
diabetes
0    1373
1     127
Name: count, dtype: int64

students:
Target
Graduate    913
Dropout     587
Name: count, dtype: int64

telco:
Churn
No     1102
Yes     398
Name: count, dtype: int64

pistachio:
Class
Kirmizi_Pistachio    860
Siirt_Pistachio      640
Name: count, dtype: int64



Sprawdzanie wartości zmiennyck o typach 'object'

In [9]:
dataset = datasets_wanted_size.get('diabetes')
object_columns = dataset.select_dtypes(include='object')
object_columns_unique_values = {column: dataset[column].unique() for column in object_columns}

for key in object_columns_unique_values:
    print(f'{key}: {object_columns_unique_values.get(key)}')

dataset.head(3)

# nulls :
    # czy w gender: 'Other'==null
    # czy w smoking_history 'No info'==null

# object column encoding :
    # gender - 
    # smoking_history - 



gender: ['Male' 'Female']
smoking_history: ['former' 'never' 'ever' 'No Info' 'not current' 'current']


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
60094,Male,62.0,1,1,former,27.32,4.8,155,0
90264,Male,61.0,0,0,never,58.05,5.7,155,1
53832,Female,73.0,0,0,never,27.32,6.5,200,0


In [10]:
dataset = datasets_wanted_size.get('students')
object_columns = dataset.select_dtypes(include='object')
object_columns_unique_values = {column: dataset[column].unique() for column in object_columns}

for key in object_columns_unique_values:
    print(f'{key}: {object_columns_unique_values.get(key)}')

# kolumny są numeryczne

dataset.head(3)

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
4310,1,15,1,6,1,14,1,23,27,10,...,1,6,12,6,13.625,0,12.4,0.5,1.79,1
4318,1,1,1,2,1,1,14,1,1,10,...,4,11,12,11,15.000,0,8.9,1.4,3.51,1
209,1,12,1,11,1,12,1,3,14,3,...,0,6,6,5,12.800,0,16.2,0.3,-0.92,0


In [11]:
dataset = datasets_wanted_size.get('telco')
object_columns = dataset.select_dtypes(include='object')
object_columns_unique_values = {column: dataset[column].unique() for column in object_columns}

for key in object_columns_unique_values:
    print(f'{key}: {object_columns_unique_values.get(key)}')
    
dataset.head(3)

# object column encoding :
    # wszytkie kolumny wartościach binarnych - label encoding
    # Contract - 
    # TotalCharges - zmiana typu na float, (są wartości ' ' ,które mogą być traktowane jako null)
    # pozostałe - one-hot encoding

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['Yes' 'No' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['Yes' 'No' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['Yes' 'No' 'No internet service']
Contract: ['Two year' 'Month-to-month' 'One year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4726,Female,0,Yes,No,72,No,No phone service,DSL,Yes,Yes,Yes,No,No,Yes,Two year,Yes,Electronic check,49.20,3580.95,0
5892,Male,0,No,No,10,Yes,No,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Mailed check,51.65,524.50,0
274,Male,0,No,No,1,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,No,Mailed check,70.60,70.60,1


In [12]:
dataset = datasets_wanted_size.get('pistachio')
object_columns = dataset.select_dtypes(include='object')
object_columns_unique_values = {column: dataset[column].unique() for column in object_columns}

for key in object_columns_unique_values:
    print(f'{key}: {object_columns_unique_values.get(key)}')

# kolumny są numeryczne

dataset.head(3)

,Area,Perimeter,Major_Axis,Minor_Axis,Eccentricity,Eqdiasq,Solidity,Convex_Area,Extent,Aspect_Ratio,...,StdDev_RR,StdDev_RG,StdDev_RB,Skew_RR,Skew_RG,Skew_RB,Kurtosis_RR,Kurtosis_RG,Kurtosis_RB,Class
1682,80139,1075.2321,418.1198,244.6082,0.8110,319.4310,0.9928,80719,0.7494,1.7093,...,14.5330,15.2768,14.2298,-0.7579,-0.6986,-0.3487,3.4817,3.2926,2.6470,1
89,85969,2345.6279,421.6958,291.1149,0.7235,330.8461,0.8702,98789,0.7033,1.4486,...,18.2206,18.1049,16.7189,-0.9762,-0.9983,-0.7881,3.1975,3.3114,3.3104,0
2021,95334,1251.7111,488.1582,249.2888,0.8598,348.4007,0.9858,96705,0.7721,1.9582,...,23.4085,24.2855,22.0737,-0.6492,-0.4470,-0.4217,2.6227,2.3029,2.2124,1


# Preprocessing

- jeżeli kolumna ma 2 wartości to label encoding, wpp one-hot encoding ,(może coś innego pomyśleć) - wzialem dummy onehotencoder do pipeline
- 

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector

In [14]:
# notatnik
# diabetes.nunique()==2

# pip_num = Pipeline(steps=[('impute',SimpleImputer()),
#                           ('scale',MinMaxScaler())])
# pip_cat = Pipeline(steps=[
#     ('ohe',OneHotEncoder()),
#     ('impute',SimpleImputer())])

# col_trans = ColumnTransformer([
#     ("numeric",pip_num,make_column_selector(dtype_include= np.number)),
#     ("cat",pip_cat,make_column_selector(dtype_include= np.object_))
# ])

# model_pipeline = Pipeline([("trans",col_trans),("model",RandomForestClassifier())])
# model_pipeline.fit(datasets_X_test.get('telco'),np.array(datasets_y_test.get('telco')))

# Modele

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [16]:
# 1 - DecisionTreeClassifier

# class sklearn.tree.DecisionTreeClassifier(*, 
#                                         criterion='gini',
#                                         splitter='best',
#                                         max_depth=None,
#                                         min_samples_split=2, 
#                                         min_samples_leaf=1, 
#                                         min_weight_fraction_leaf=0.0, 
#                                         max_features=None, 
#                                         random_state=None, 
#                                         max_leaf_nodes=None, 
#                                         min_impurity_decrease=0.0, 
#                                         class_weight=None, 
#                                         ccp_alpha=0.0, 
#                                         monotonic_cst=None)


In [17]:
# 2 - LogisticRegression

# class sklearn.linear_model.LogisticRegression(penalty='l2', 
#                                               *, 
#                                               dual=False, 
#                                               tol=0.0001, 
#                                               C=1.0, 
#                                               fit_intercept=True, 
#                                               intercept_scaling=1, 
#                                               class_weight=None, 
#                                               random_state=None, 
#                                               solver='lbfgs', 
#                                               max_iter=100, 
#                                               multi_class='deprecated', 
#                                               verbose=0, 
#                                               warm_start=False, 
#                                               n_jobs=None, 
#                                               l1_ratio=None)

In [18]:
# 3 - GradientBoostingClassifier

# class sklearn.ensemble.GradientBoostingClassifier(*, 
#                                                   loss='log_loss', 
#                                                   learning_rate=0.1, 
#                                                   n_estimators=100, 
#                                                   subsample=1.0, 
#                                                   criterion='friedman_mse', 
#                                                   min_samples_split=2, 
#                                                   min_samples_leaf=1, 
#                                                   min_weight_fraction_leaf=0.0, 
#                                                   max_depth=3, 
#                                                   min_impurity_decrease=0.0, 
#                                                   init=None, random_state=None, 
#                                                   max_features=None, 
#                                                   verbose=0, 
#                                                   max_leaf_nodes=None, 
#                                                   warm_start=False, 
#                                                   validation_fraction=0.1, 
#                                                   n_iter_no_change=None, 
#                                                   tol=0.0001, 
#                                                   ccp_alpha=0.0)

In [19]:
# Import the required classifiers from scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Create objects of the classifiers
logistic_regression = LogisticRegression()
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()
gradient_boosting = GradientBoostingClassifier()
ada_boost = AdaBoostClassifier()
bagging_classifier = BaggingClassifier()
k_neighbors = KNeighborsClassifier()
support_vector_machine = SVC()

# Display the classifier objects
classifiers = {
    "Logistic Regression": logistic_regression,
    "Decision Tree": decision_tree,
    "Random Forest": random_forest,
    "Gradient Boosting": gradient_boosting,
    "AdaBoost": ada_boost,
    "Bagging Classifier": bagging_classifier,
    "K-Neighbors": k_neighbors,
    "Support Vector Machine": support_vector_machine
}

In [20]:
random_seed = 24
datasets_X_train = {}
datasets_X_test = {}
datasets_y_train = {}
datasets_y_test = {}
for key in datasets_wanted_size:
    dataset_size = len(datasets_wanted_size.get(key))
    X = datasets_wanted_size.get(key).iloc[:,:-1]
    y = datasets_wanted_size.get(key).iloc[:,-1]
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=random_seed)
    datasets_X_train[key] = X_train
    datasets_X_test[key] = X_test
    datasets_y_train[key] = y_train
    datasets_y_test[key] = y_test

In [21]:
from timeit import default_timer as timer
tabela_wyników = {}

pip_num = Pipeline(steps=[('impute',SimpleImputer()),
                          ('scale',MinMaxScaler())])
pip_cat = Pipeline(steps=[
    ('ohe',OneHotEncoder(drop='first')),
    ('impute',SimpleImputer())])
col_trans = ColumnTransformer([
    ("numeric",pip_num,make_column_selector(dtype_include= np.number)),
    ("cat",pip_cat,make_column_selector(dtype_include= np.object_))
])


example_pipeline = Pipeline([("trans",col_trans),("model",RandomForestClassifier())])
example_pipeline.fit(datasets_X_test.get('telco'),np.array(datasets_y_test.get('telco')))



Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000025298FC2FC0>),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first')),
                                                                  ('impute',
                                                                   SimpleImputer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000025298FC2780>)])),
                ('model', RandomForestClassifier())])

In [ ]:
set_key = 'students'

for key in classifiers:
    start = timer()
    model_pipeline = Pipeline([("trans",col_trans),("model",classifiers[key])])
    model_pipeline.fit(datasets_X_train.get(set_key),datasets_y_train.get(set_key))    
    end = timer()
    y_hat = model_pipeline.predict(datasets_X_test.get(set_key))
    tabela_wyników[key] = [accuracy_score(y_hat,datasets_y_test.get(set_key)),end-start] #== [accuracy, time]


# pistachio
# {'Logistic Regression': [0.8766666666666667, 0.014618600020185113],
#  'Decision Tree': [0.8066666666666666, 0.02735049999319017],
#  'Random Forest': [0.8533333333333334, 0.3635247000493109],
#  'Gradient Boosting': [0.8666666666666667, 0.9874855000525713],
#  'AdaBoost': [0.8933333333333333, 0.2399780999403447],
#  'Bagging Classifier': [0.8466666666666667, 0.16180030000396073],
#  'K-Neighbors': [0.8766666666666667, 0.004990700050257146],
#  'Support Vector Machine': [0.89, 0.021320700063370168]}

# diabetes
# {'Logistic Regression': [0.96, 0.02030680002644658],
#  'Decision Tree': [0.94, 0.007484599947929382],
#  'Random Forest': [0.98, 0.11521570000331849],
#  'Gradient Boosting': [0.97, 0.14825059997383505],
#  'AdaBoost': [0.97, 0.078939100028947],
#  'Bagging Classifier': [0.97, 0.02978769992478192],
#  'K-Neighbors': [0.9266666666666666, 0.00779770000372082],
#  'Support Vector Machine': [0.96, 0.011258100043050945]}

# telco
# {'Logistic Regression': [0.8033333333333333, 0.024130700039677322],
#  'Decision Tree': [0.7566666666666667, 0.015992800006642938],
#  'Random Forest': [0.8, 0.15928849996998906],
#  'Gradient Boosting': [0.7966666666666666, 0.24993829999584705],
#  'AdaBoost': [0.8066666666666666, 0.10428309999406338],
#  'Bagging Classifier': [0.7833333333333333, 0.04923330002930015],
#  'K-Neighbors': [0.75, 0.011039600009098649],
#  'Support Vector Machine': [0.81, 0.028400499955751002]}

# students 
# {'Logistic Regression': [0.9066666666666666, 0.011891300091519952],
#  'Decision Tree': [0.8666666666666667, 0.01138309994712472],
#  'Random Forest': [0.9166666666666666, 0.1611720999935642],
#  'Gradient Boosting': [0.9233333333333333, 0.2623509999830276],
#  'AdaBoost': [0.9133333333333333, 0.12784590001683682],
#  'Bagging Classifier': [0.8966666666666666, 0.054303099983371794],
#  'K-Neighbors': [0.8233333333333334, 0.00486270000692457],
#  'Support Vector Machine': [0.8833333333333333, 0.0196890999795869]}


In [21]:
tabela_wyników

{'Logistic Regression': [0.9066666666666666, 0.016913700033910573],
 'Decision Tree': [0.87, 0.01254030002746731],
 'Random Forest': [0.9266666666666666, 0.15005080006085336],
 'Gradient Boosting': [0.9266666666666666, 0.2406470999121666],
 'AdaBoost': [0.9133333333333333, 0.0902577000670135],
 'Bagging Classifier': [0.9066666666666666, 0.055144599988125265],
 'K-Neighbors': [0.8233333333333334, 0.004798400099389255],
 'Support Vector Machine': [0.8833333333333333, 0.031351299956440926]}

# Wybrane modele

In [142]:
from sklearn.svm import SVC
from sklearn.linear_model import ElasticNet

models = {
    # 'Elastic Net' : ElasticNet,
    "KNN" : KNeighborsClassifier,
    'Decision Tree' : DecisionTreeClassifier,
    # 'Random Forest' : RandomForestClassifier,
    'Logistic Regression' : LogisticRegression
}

# Random Search

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [48]:
# from sklearn.model_selection import RandomizedSearchCV
# random_forest_grid = {
#     'n_estimators': [i for i in range(1,2001)],
#     'max_depth': [i for i in range(5, 31)],
#     'min_samples_split': [i for i in range(2, 11)],
#     'min_samples_leaf': [i for i in range(1,11)],
#     'max_features': [None, 'sqrt', 'log2'],
#     'bootstrap': [True, False],
#     'criterion': ['gini', 'entropy']
# }

# svm_grid = {
#     'C': [0.001,0.01,0.1] + [i for i in range(1,1000)],
#     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],    
#     'degree': [2, 3, 4, 5],                             
#     'gamma': [float(10 ** i) for i in np.arange(-5, 5.1, 0.1)],  
#     'shrinking': [True, False],                        
#     'class_weight': [None, 'balanced'],                
# }

# elastic_net_grid = {
#     'alpha': [i for i in np.arange(-5, 5.1, 0.1)],  
#     'l1_ratio': [i for i in np.arange(-5, 5.1, 0.1)], 
#     }


In [130]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Random Forest Grid
random_forest_grid = {
    'model__n_estimators': [i for i in range(1, 2001)],  # Number of trees
    'model__max_depth': [None] + [i for i in range(5, 31)],  # Allow for unlimited depth as an option
    'model__min_samples_split': [i for i in range(2, 11)],  # Minimum samples to split an internal node
    'model__min_samples_leaf': [i for i in range(1, 11)],   # Minimum samples at a leaf node
    'model__max_features': [None, 'sqrt', 'log2'],           # Number of features to consider for best split
    'model__bootstrap': [True, False],                        # Whether bootstrap samples are used
    'model__criterion': ['gini', 'entropy']                   # Function to measure the quality of a split
}
# Decision tree grid
decision_tree_grid = {
    'model__criterion': ['gini', 'entropy'],                    # Quality of split measure
    'model__max_depth': [None] + [i for i in range(1, 31)],     # Maximum depth, with None for no limit
    'model__min_samples_split': [i for i in range(2, 61)],      # Minimum samples required to split a node
    'model__min_samples_leaf': [i for i in range(1, 61)],       # Minimum samples required at a leaf node
    'model__max_features': [None, 'sqrt', 'log2'],              # Number of features to consider for the best split
    'model__max_leaf_nodes': [None] + [i for i in range(10, 101, 10)],  # Max number of terminal nodes
    'model__splitter': ['best', 'random']                       # Strategy to choose the split at each node
}

# SVM Grid
svm_grid = {
    'model__C': [0.001, 0.01, 0.1] + [i for i in range(1, 1001,2)],  # Regularization parameter
    'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],        # Specifies the kernel type
    'model__degree': [2, 3, 4, 5],                                # Degree of the polynomial kernel
    'model__gamma': [float(10 ** i) for i in np.arange(-5, 5.1, 0.1)],  # Kernel coefficient
    'model__shrinking': [True, False],                            # Whether to use the shrinking heuristic
    'model__class_weight': [None, 'balanced'],                   # Weights associated with classes
}

# # Elastic Net Grid
# elastic_net_grid = {
#     'model__alpha': [10 ** i for i in np.arange(-5, 5.1, 0.1)],  # Regularization parameter (positive values)
#     'model__l1_ratio': [i / 10 for i in range(0, 11)],           # l1_ratio between 0 and 1
# }

logistic_regression_grid = {
    'model__C': np.logspace(-4, 4, num=9),  # Values from 0.0001 to 10000 (logarithmically spaced)
    'model__penalty': ['l1', 'l2', 'elasticnet','None'],  # Types of regularization
    'model__solver': ['liblinear', 'saga', 'lbfgs'],  # Supported solvers (note that 'elasticnet' requires 'saga')
    'model__max_iter': [100, 200, 300, 400, 500],  # Number of iterations for convergence
    'model__class_weight': [None, 'balanced'],  # Weighting of classes
}



knn_grid = {
    'model__n_neighbors': [i for i in range(1, 31)],           # Liczba sąsiadów (1 do 30)
    'model__p': [1, 2, 3],                                     # Parametr dla metryki Minkowskiego (1: Manhattan, 2: Euklidesowa, 3: Minkowski)
    'model__weights': ['uniform', 'distance'],                 # Waga (jednolita lub zależna od odległości)
    'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  # Algorytm do wyszukiwania sąsiadów
}


In [97]:
best_random_dt = {'diabetes': None,'students': None,'telco': None,'pistachio': None}
best_random_knn = {'diabetes': None,'students': None,'telco': None,'pistachio': None}
best_random_en = {'diabetes': None,'students': None,'telco': None,'pistachio': None}
best_random_rf = {'diabetes': None,'students': None,'telco': None,'pistachio': None}
best_random_svm = {'diabetes': None,'students': None,'telco': None,'pistachio': None}
best_random_lr = {'diabetes': None,'students': None,'telco': None,'pistachio': None}
n_iter = 10


In [87]:
# random forest
# random_search = RandomizedSearchCV(
#     estimator= Pipeline([("trans",col_trans),("model",models['Random Forest']())]),
#     param_distributions=random_forest_grid,
#     n_iter=n_iter,  
#     cv=3,        
#     random_state=42,  
# )
# for dataset in datasets_wanted_size.keys():
#     random_search.fit(datasets_X_train[dataset], datasets_y_train[dataset])
#     best_rf[dataset] = random_search.best_estimator_
    
# za długo sie liczy

In [ ]:
# # svm
# random_search = RandomizedSearchCV(
#     estimator= Pipeline([("trans",col_trans),("model",models['SVM']())]) ,
#     param_distributions=svm_grid,
#     n_iter=n_iter,  
#     cv=3,        
#     random_state=42,  
# )
# for dataset in datasets_wanted_size.keys():
#     random_search.fit(datasets_X_train[dataset], datasets_y_train[dataset])
#     best_svm[dataset] = random_search.best_estimator_

# za dlugo sie liczy

In [145]:
# KNN

random_search_knn = RandomizedSearchCV(
    estimator= Pipeline([("trans",col_trans),("model",models['KNN']())]),
    param_distributions=knn_grid,
    n_iter=n_iter,  
    cv=3,        
    random_state=42,  
)
for dataset in datasets_wanted_size.keys():
    random_search_knn.fit(datasets_X_train[dataset], datasets_y_train[dataset])
    best_random_knn[dataset] = random_search_knn.best_estimator_

In [146]:
# Decision Tree
random_search_dt = RandomizedSearchCV(
    estimator= Pipeline([("trans",col_trans),("model",models['Decision Tree']())]),
    param_distributions=decision_tree_grid,
    n_iter=n_iter,  
    cv=3,        
    random_state=42,  
)
for dataset in datasets_wanted_size.keys():
    random_search_dt.fit(datasets_X_train[dataset], datasets_y_train[dataset])
    best_random_dt[dataset] = random_search_dt.best_estimator_

# 3 sekundy dla 10 iteracji

In [61]:
# elastic net
# random_search = RandomizedSearchCV(
#     estimator=Pipeline([("trans",col_trans),("model",models['Elastic Net']())]),
#     param_distributions=elastic_net_grid,
#     n_iter=n_iter,  
#     cv=3,        
#     random_state=42,  
# )
# for dataset in datasets_wanted_size.keys():
#     random_search.fit(datasets_X_train[dataset], datasets_y_train[dataset])
#     best_en[dataset] = random_search.best_estimator_

# 24 sekundy dla 10 kombinacji hiperparametrów

In [147]:
# logistic regression
random_search_lr = RandomizedSearchCV(
    estimator=Pipeline([("trans",col_trans),("model",models['Logistic Regression']())]),
    param_distributions=logistic_regression_grid,
    n_iter=n_iter,  
    cv=3,        
    random_state=42,  
)
for dataset in datasets_wanted_size.keys():
    random_search_lr.fit(datasets_X_train[dataset], datasets_y_train[dataset])
    best_random_lr[dataset] = random_search_lr.best_estimator_

# 15 sekund dla 10 iteracji

In [132]:
accuracy_score(best_random_lr['students'].predict(datasets_X_test['students']) , datasets_y_test['students'])

0.9266666666666666

# Bayes Search

In [ ]:
# opt = BayesSearchCV(
#     pipe,
#     # (parameter space, # of evaluations)
#     [(svc_search, 40), (linsvc_search, 16)],
#     cv=3
# )

In [101]:
from skopt import BayesSearchCV
best_bayes_knn = {'diabetes': None,'students': None,'telco': None,'pistachio': None}
best_bayes_rf = {'diabetes': None,'students': None,'telco': None,'pistachio': None}
best_bayes_lr = {'diabetes': None,'students': None,'telco': None,'pistachio': None}

In [140]:
# logistic regression

bayes_search_lr = BayesSearchCV(
    estimator = Pipeline([("trans",col_trans),("model",models['Logistic Regression']())]),
    # (parameter space, # of evauations)
    search_spaces = logistic_regression_grid,
    scoring = 'accuracy', # ?
    cv=3
)

for dataset in datasets_wanted_size.keys():
    bayes_search_lr.fit(datasets_X_train[dataset], datasets_y_train[dataset])
    best_bayes_lr[dataset] = random_search.best_estimator_

ValueError: l1_ratio must be specified when penalty is elasticnet.

In [141]:
# Próba naprawienia errora

# Tworzenie funkcji do filtrowania parametrów
def filter_param_grid(param_grid):
    valid_grid = {
        'model__C': param_grid['model__C'],
        'model__penalty': [],
        'model__solver': [],
        'model__max_iter': param_grid['model__max_iter'],
        'model__class_weight': param_grid['model__class_weight'],
    }
    
    for penalty in param_grid['model__penalty']:
        for solver in param_grid['model__solver']:
            if (penalty == 'l1' and solver in ['liblinear', 'saga']) or \
               (penalty == 'l2' and solver in ['liblinear', 'lbfgs', 'saga']) or \
               (penalty == 'elasticnet' and solver == 'saga'):
                valid_grid['model__penalty'].append(penalty)
                valid_grid['model__solver'].append(solver)

    return valid_grid

# Filtrowanie siatki
filtered_grid = filter_param_grid(logistic_regression_grid)

# Model tunability

In [163]:
from sklearn.metrics import log_loss

criterion = log_loss

best_estimators_random = {'Decision Tree':best_random_dt,"KNN": best_random_knn,"Logistic Regression":best_random_lr}
best_scores_random = {}
deafult_scores = {}
tunability_random = {}

In [174]:
# loss dla najepszych modeli

for model in models.keys():
    best_scores_random[model] = {}
    for dataset in datasets_wanted_size.keys():
        best_scores_random[model][dataset] =  criterion(datasets_y_test[dataset],best_estimators_random[model][dataset].predict(datasets_X_test[dataset]))

print(best_scores_random['Decision Tree'])

{'diabetes': 0.7208730677823433, 'students': 3.364074316317601, 'telco': 7.929603745605775, 'pistachio': 6.127421076149917}


In [173]:
# loss dla deafult modeli

for model in models.keys():
    deafult_model = Pipeline([("trans",col_trans),("model",models[model]())])
    deafult_scores[model] = {}
    for dataset in datasets_wanted_size.keys():
        deafult_model.fit(datasets_X_train[dataset] , datasets_y_train[dataset])
        deafult_scores[model][dataset] =  criterion(datasets_y_test[dataset] , deafult_model.predict(datasets_X_test[dataset]))

print(deafult_scores['Decision Tree'])


{'diabetes': 2.1626192033470293, 'students': 4.925965963179343, 'telco': 9.731786415061631, 'pistachio': 7.208730677823431}


In [175]:
# Tunability dla poszczególnych datasetów
# Łączna dla modelu to średnia? idk trzeba bedzie dopytac


for model in models.keys():
    tunability_random[model] = {}
    for dataset in datasets_wanted_size.keys():
        tunability_random[model][dataset] = deafult_scores[model][dataset] - best_scores_random[model][dataset]

print(tunability_random['Decision Tree'])

{'diabetes': 1.441746135564686, 'students': 1.561891646861742, 'telco': 1.8021826694558563, 'pistachio': 1.0813096016735146}


In [ ]:
#To samo dla bayesa trzeba bedzie